In [32]:
import warnings
warnings.filterwarnings('ignore')

import random
import numpy as np
import pandas as pd

# This part is to Prepare the Senetence Selection Data Set Using the Samplling Method Mentioned in the report
No need to run this it is just for demonstration purpose it requires running another notebook

In [ ]:
def extract_final_text_from_line(line):
    # use this function to extarct the text from a line in data-file
    #try:
    line_list =line.split(" ")
    page_id = line_list.pop(0)
    page_name = " ".join(page_id.split("_"))
    sentence_id = 12345
    if not line_list:
        return " "
    if line_list[0].isnumeric():
        sentence_id = line_list.pop(0)
    line = " ".join(line_list)
    return line

test = pd.read_csv("search_results_v5_test.csv")

# define the required top-k
k = 13
mask = (test["rank"] <= k)
test = test[mask]

test.head()

file = 'train_gold_retrived.json'
with open(file) as train_file:
    dev_d = json.load(train_file)

from tqdm import tqdm



final_d = {}
for key,item in tqdm(dev_d.items()):
    if item["evidence"]:
        final_d[key] = {}
        final_d[key]["claim"] = item["claim"]
        gold = set([extract_final_text_from_line(i).strip() for i in item["gold_evidence"]])
        results = set(dev[dev["claim_id"] ==int(key)].text.tolist())
        gold_retrived = results.intersection(gold)
        wrong_rem = results.difference(gold_retrived)
        final_d[key]["gold_retrived"] = list(gold_retrived)
        final_d[key]["wrong_remained"] = list(wrong_rem)

with open('dev_train_sentence.json',"w") as train_file:
    json.dump(final_d, train_file,indent=0)

label_d = []
for key, item in final_d.items():
    if item["gold_retrived"]:
        gold = item["gold_retrived"]
        wrn = item["wrong_remained"]
        for i in gold:
            label_d.append(dict(label=1,claim=item["claim"],evidence=i))
        for j in wrn:
            label_d.append(dict(label=0,claim=item["claim"],evidence=j))
        

final_df = pd.DataFrame(label_d)
label_counts = list(final_df.label.value_counts())
print(" The 1's:"," ",label_counts[1],"\n","The 0's:"," ",label_counts[0], "\n",
     "The Probability of index returing a Gold:",(label_counts[1]/(label_counts[1]+label_counts[0])) )

final_df.to_csv("dev_top13_train.csv".format(k),index=False)






In [ ]:
# Run from here 

In [42]:
df = pd.read_csv("dev_top13_train.csv")

In [43]:
df["words_len"] = df.evidence.apply(lambda x : len(x.split()))

mask = df["words_len"] < 130 
df = df[mask]

X = df[["claim","evidence"]]
y = df["label"]

Making a Stratified Split to Main Distribution

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.25)

X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train,
                                                    stratify=y_train, 
                                                    test_size=0.2)

In [45]:
y_train.value_counts()

0    19401
1     2031
Name: label, dtype: int64

In [46]:
train = pd.DataFrame()
train[["claim","evidence"]] = X_train
train["label"] = y_train
dev = pd.DataFrame()
dev[["claim","evidence"]] = X_dev
dev["label"] = y_dev
test = pd.DataFrame()
test[["claim","evidence"]] = X_test
test["label"] = y_test

In [47]:
test.to_csv('./bert/data/test.csv', index=False, header=True)

In [48]:
train = train.reset_index().rename(columns={"index":"id"})
dev = dev.reset_index().rename(columns={"index":"id"})
test = test.reset_index().rename(columns={"index":"id"})

In [49]:
train["id1"] = train.id
dev["id1"] = dev.id
test["id1"] = test.id
test["index"] =test.index

Tuning into MRPC Data(GLUE) format

In [50]:
columns = ['label','id',"id1",'claim', 'evidence']
rename = {'label':"Quality","id":"#1 ID","id1":"#2 ID","claim":"#1 String","evidence":"#2 String"}

In [51]:
train = train[columns]
dev = dev[columns]
test = test[["index",'id',"id1",'claim', 'evidence']]

In [52]:
train.rename(columns=rename,inplace=True)
dev.rename(columns=rename,inplace=True)
test.rename(columns=rename,inplace=True)

In [53]:
train.to_csv('./bert/data/train.tsv', sep='\t', index=False, header=True)
dev.to_csv('./bert/data/dev.tsv', sep='\t', index=False, header=True)
#test.to_csv('./bert/data/test.tsv', sep='\t', index=False, header=True)

In [54]:
test.head()

,index,#1 ID,#2 ID,#1 String,#2 String
0,0,5490,5490,Nuuk is a seat of government.,Nuuk Cathedral -LRB- Annaassisitta Oqaluffia -...
1,1,30834,30834,North Vietnam was officially called another name.,Another tournament named Vietnam International...
2,2,954,954,Drake Bell put out an extended play in 2011.,"Born in Cincinnati , Ohio , he played collegia..."
3,3,28146,28146,Down With Love is a romantic movie.,Down to You is a 2000 American romantic comedy...
4,4,14548,14548,Dilwale Dulhania Le Jayenge began filming in C...,Earning 1.06 billion -LRB- valued at about US$...


In [55]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import numpy as np

# Actual Test Part

In [7]:
import json
import pandas as pd
import numpy as np

In [8]:
file = 'test-unlabelled.json'
with open(file) as train_file:
    tst = json.load(train_file)

In [33]:
file = 'train.json'
with open(file) as train_file:
    dev = json.load(train_file)

In [14]:
test_df = pd.read_csv('data/search_result_v5_test.csv')

In [15]:
test_df["claim"] = test_df.claim_id.apply(lambda x: tst[str(x)]["claim"])

In [16]:
test_df.rename(columns={"text":"evidence"},inplace=True)

In [17]:
test_df.dropna().shape

(500100, 8)

In [18]:
# Get the top-k desired
k = 13

In [19]:
mask = (test_df["rank"] <=k)
test_df = test_df[mask]

In [20]:
test_df.shape

(125025, 8)

In [21]:
test_df["evidence"] = test_df["evidence"].apply(lambda x: x.replace("\n","").strip())

In [22]:
test_df.head()

,claim_id,found_doc,sentence_id,evidence,rank,percentage,weight,claim
0,12,Carlos_Santana_Live,0,Carlos Santana Live is a 2004 live album by gu...,1,50,40.007783,Carlos Santana disbanded Santana in 1965.
1,12,Coke_Escovedo,19,During Carlos Santana 's transition period bet...,2,47,37.767251,Carlos Santana disbanded Santana in 1965.
2,12,Jorge_Santana,17,In 1994 he recorded an album with his brother ...,3,47,37.698147,Carlos Santana disbanded Santana in 1965.
3,12,Santana,2,"Santana -LRB- band -RRB- , rock band featuring...",4,46,37.459342,Carlos Santana disbanded Santana in 1965.
4,12,Carlos_Alves,5,Carlos Santana -LRB- Carlos Augusto Alves Sant...,5,46,37.210843,Carlos Santana disbanded Santana in 1965.


In [23]:
test_df.to_csv('./bert/data/test.csv', index=False, header=True)

In [24]:
test_df = test_df.rename(columns={"claim_id":"id"})

In [25]:
test_df["id1"] = test_df.id
test_df["index"] =test_df.index

In [26]:
columns = ["index",'id',"id1",'claim', 'evidence']
rename = {'label':"Quality","id":"#1 ID","id1":"#2 ID","claim":"#1 String","evidence":"#2 String"}

In [27]:
test_df = test_df[columns]

In [28]:
test_df.rename(columns=rename,inplace=True)

In [29]:
test_df.head()

,index,#1 ID,#2 ID,#1 String,#2 String
0,0,12,12,Carlos Santana disbanded Santana in 1965.,Carlos Santana Live is a 2004 live album by gu...
1,1,12,12,Carlos Santana disbanded Santana in 1965.,During Carlos Santana 's transition period bet...
2,2,12,12,Carlos Santana disbanded Santana in 1965.,In 1994 he recorded an album with his brother ...
3,3,12,12,Carlos Santana disbanded Santana in 1965.,"Santana -LRB- band -RRB- , rock band featuring..."
4,4,12,12,Carlos Santana disbanded Santana in 1965.,Carlos Santana -LRB- Carlos Augusto Alves Sant...


In [30]:
test_df.to_csv('./bert/data/test.tsv', sep='\t', index=False, header=True)

In [31]:
test_df.shape

(125025, 5)

In [21]:
test_df.head()

,index,#1 ID,#2 ID,#1 String,#2 String
0,0,12,12,Carlos Santana disbanded Santana in 1965.,Carlos Santana Live is a 2004 live album by gu...
1,1,12,12,Carlos Santana disbanded Santana in 1965.,During Carlos Santana 's transition period bet...
2,2,12,12,Carlos Santana disbanded Santana in 1965.,In 1994 he recorded an album with his brother ...
3,3,12,12,Carlos Santana disbanded Santana in 1965.,"Santana -LRB- band -RRB- , rock band featuring..."
4,4,12,12,Carlos Santana disbanded Santana in 1965.,Carlos Santana -LRB- Carlos Augusto Alves Sant...


Reading and Generating the predictions file for Sentence pairs

In [72]:
df = pd.read_csv('./bert/data/test.csv')

In [73]:
df.shape

(1502, 7)

# Adding Label Prediction to CSV Running this part after tuning BERT 

In [34]:
test_o = pd.read_csv('./bert/data/test.csv')

test_pred = pd.read_csv('./bert/bert_output/test_results.tsv', sep='\t',header=None)

y_pred = np.argmax(np.array(test_pred[[0,1]]), axis=1)

In [35]:
assert test_o.shape[0] == y_pred.shape[0]

In [36]:
mask = y_pred.astype(bool)

In [37]:
test = test_o[mask]

In [38]:
test_o.head()

,claim_id,found_doc,sentence_id,evidence,rank,percentage,weight,claim
0,12,Carlos_Santana_Live,0,Carlos Santana Live is a 2004 live album by gu...,1,50,40.007783,Carlos Santana disbanded Santana in 1965.
1,12,Coke_Escovedo,19,During Carlos Santana 's transition period bet...,2,47,37.767251,Carlos Santana disbanded Santana in 1965.
2,12,Jorge_Santana,17,In 1994 he recorded an album with his brother ...,3,47,37.698147,Carlos Santana disbanded Santana in 1965.
3,12,Santana,2,"Santana -LRB- band -RRB- , rock band featuring...",4,46,37.459342,Carlos Santana disbanded Santana in 1965.
4,12,Carlos_Alves,5,Carlos Santana -LRB- Carlos Augusto Alves Sant...,5,46,37.210843,Carlos Santana disbanded Santana in 1965.


In [39]:
test.claim_id.nunique()

4124

In [40]:
test.shape

(6088, 8)

In [41]:
test.to_csv("test_pred_top13.csv",index=False)